In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#import tushare as ts
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io as scio
import time 

In [2]:
path_noisy = r'Input_Patches_2Dsyn11.csv'

In [3]:
data = pd.read_csv(path_noisy, header=None)

In [4]:
#type(data.shape[1])

In [5]:
data = data.T

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,-0.285655,-0.094759,0.059033,0.082095,-0.169731,-0.141456,-0.065269,-0.101775,-0.305867,0.142491,...,0.010872,-0.494745,0.326118,0.023124,0.054619,0.011072,0.328060,-0.203638,-0.329366,0.017071
1,-0.034454,0.024164,-0.350245,0.085406,0.066838,0.103323,0.183920,0.196916,0.400856,-0.097729,...,-0.083210,-0.213278,0.398580,0.199259,0.193139,-0.182990,-0.151285,0.054522,0.065290,0.144964
2,-0.079837,-0.129883,-0.157144,-0.011358,0.043089,-0.154546,-0.025915,0.608872,0.222907,-0.214785,...,0.482898,0.356052,-0.223782,-0.116979,0.066285,-0.001133,-0.052318,-0.062070,-0.075138,0.269355
3,0.239533,-0.248196,0.082898,-0.101518,0.311064,-0.005159,0.091698,0.111151,-0.051184,-0.204132,...,-0.314867,-0.242084,-0.071824,-0.056087,0.134982,0.389893,0.016835,-0.232038,0.160488,0.108543
4,0.625598,0.099131,0.100790,0.496220,-0.033520,1.222121,0.216445,0.203567,0.363440,-0.184805,...,0.282989,0.312111,0.119921,-0.299165,0.060922,0.158843,-0.100142,0.015174,0.059186,0.354470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33137,0.016760,0.095579,0.179706,0.077812,0.106669,-0.449320,0.047308,-0.429691,0.127659,0.050212,...,-0.036615,-0.164099,-0.182914,-0.083881,0.217836,-0.162975,-0.216453,0.528084,0.151361,-0.348705
33138,-0.231164,-0.173054,-0.117294,0.149689,-0.097406,0.342493,-0.011129,-0.307579,-0.147544,0.015597,...,-0.239727,0.168824,0.329842,-0.226346,0.250965,0.353435,0.181593,0.019157,-0.116774,-0.007252
33139,-0.078532,0.020383,-0.073423,0.043979,-0.008138,0.102078,-0.326204,-0.343582,-0.218178,0.136061,...,-0.076938,0.521511,0.112031,-0.056174,-0.225504,0.015828,-0.083210,0.146706,0.143406,0.167758
33140,-0.231931,0.061216,-0.168617,-0.094112,-0.081586,-0.113479,0.063115,-0.027429,-0.097478,-0.018648,...,-0.258984,-0.148469,-0.149496,0.053072,-0.309760,0.048369,-0.032407,-0.070822,-0.263769,0.302118


In [7]:
data = data.astype(np.float32)
data= torch.from_numpy(data.values)

In [8]:
data.shape

torch.Size([33142, 2304])

In [9]:
#将前80%作为训练集，后20%作为测试集
train_size = int(len(data) * 0.8)
train = data[:train_size]
vaild = data[train_size:]
print(train.shape)
print(vaild.shape)
batch_size1 = 64
w1 = 48
w2 = 48

train_data= TensorDataset(train)
vaild_data= TensorDataset(vaild)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

vaild_loader = torch.utils.data.DataLoader(vaild_data,
                                          batch_size = 64,
                                          shuffle = True)

torch.Size([26513, 2304])
torch.Size([6629, 2304])


In [10]:
#定义Fully connected (FC) block
class FCB(nn.Module):
    def __init__(self, input_size, output_size, dropout=0.3):
        super().__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        self.activation = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x) 
        x = self.bn(x)
        x = self.dropout(x)
        
        return x

In [11]:
class PAB(nn.Module):  ##position attention block
    def __init__(self, input_size, output_size, dropout=0.3):
        super().__init__()
        
        self.fcb1 = FCB(input_size, output_size, dropout)
        self.fcb2 = FCB(input_size, output_size, dropout)
        self.fcb3 = FCB(input_size, output_size, dropout)
        self.fcb4 = FCB(input_size, output_size, dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x1 = self.fcb1(x)
        x2 = self.fcb2(x)
        x3 = self.fcb3(x)
        x4 = self.fcb4(x)
        

        x = x1*x2
        x = self.softmax(x)
        x = x*x3
        x = x+x4
        
        return x   

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, dropout=0.3):
        super().__init__()

        self.cov1d = nn.Conv1d(in_channels=1, out_channels= 64 ,kernel_size = 5, padding=0, stride=1)

        self.pab1 = PAB(int(input_size-5+1), 128, dropout)
        self.fcb1 = FCB(128, 128, dropout)

        self.pab2 = PAB(128, 64, dropout)
        self.fcb2 = FCB(64, 64, dropout)

        self.pab3 = PAB(64, 32, dropout)
        self.fcb3 = FCB(32, 32, dropout)

        self.pab4 = PAB(32, 16, dropout)
        self.fcb4 = FCB(16, 16, dropout)

        self.pab5 = PAB(16, 8, dropout)
        self.fcb5 = FCB(8, 8, dropout)

        self.pab6 = PAB(8, 4, dropout)
        self.fcb6 = FCB(4, 4, dropout)
        
        self.fcb7 = FCB(4, 4, dropout)
        self.pab7 = PAB(4, 4, dropout)
        

        


    def forward(self, x):

        x = x.reshape(x.shape[0],1,x.shape[1])
        x = self.cov1d(x)
        x = x.permute(0,2,1)
        x = nn.AdaptiveAvgPool1d(1)(x)
        x = x.squeeze()

        x1 = self.pab1(x)
        x13 = self.fcb1(x1)

        x2 = self.pab2(x1)
        x12 = self.fcb2(x2)

        x3 = self.pab3(x2)
        x11 = self.fcb3(x3)

        x4 = self.pab4(x3)
        x10 = self.fcb4(x4)

        x5 = self.pab5(x4)
        x9 = self.fcb5(x5)

        x6 = self.pab6(x5)
        x8 = self.fcb6(x6)

        x7 = self.fcb7(x6)
        x7 = self.pab7(x7)
        

        

        return x7,x8,x9,x10,x11,x12,x13

In [13]:
class CustomActivation(torch.autograd.Function):    
    @staticmethod
    def forward(ctx, x):
        # 计算正向传播结果
        ctx.save_for_backward(x)
        y = 12/3 * (x - torch.tanh(x)) * torch.cos(x/2)
        return y
    
    @staticmethod
    def backward(ctx, grad_output):
        # 计算反向传播结果
        x, = ctx.saved_tensors
        grad_x = grad_output * (12/3 * ((torch.cos(x/2)*(1 - torch.pow(torch.cosh(x), -2)))+ (x - torch.tanh(x))*(-0.5*torch.sin(x/2))))
                                       
        return grad_x
    
class CustomActivationModule(torch.nn.Module):
    def forward(self, x):
        return CustomActivation.apply(x)  
    

activation = CustomActivationModule()    

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_size, dropout=0.3):
        super().__init__()

        self.pab1 = PAB(8, 8, dropout)
        self.pab2 = PAB(16, 16, dropout)
        self.pab3 = PAB(32, 32, dropout)
        self.pab4 = PAB(64, 64, dropout)
        self.pab5 = PAB(128, 128, dropout)
        #self.pab5 = PAB(256, 256, dropout)

        self.fc = nn.Linear(256,output_size)
        #self.activation = nn.Identity()
        self.activation = activation


    def forward(self, x7,x8,x9,x10,x11,x12,x13):
        
        x = torch.cat((x7,x8),dim=1)
        x = self.pab1(x)
        x = torch.cat((x,x9),dim=1)
        x = self.pab2(x)
        x = torch.cat((x,x10),dim=1)
        x = self.pab3(x)
        x = torch.cat((x,x11),dim=1)
        x = self.pab4(x)
        x = torch.cat((x,x12),dim=1)
        x = self.pab5(x)
        x = torch.cat((x,x13),dim=1)
        x = self.fc(x)
        #x.requires_grad_(True)
        x = self.activation(x)
        #x.sum().backward()
        return x

In [15]:
class AutoEncoder(nn.Module):
    def __init__(self,input_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(output_size)

    def forward(self, x):
        x7,x8,x9,x10,x11,x12,x13= self.encoder(x)
        x = self.decoder(x7,x8,x9,x10,x11,x12,x13)

        return x

In [16]:
class Loss(nn.Module):
    def __init__(self,delta,r):
        super().__init__()
        self.delta = delta
        self.r = r
    def forward(self,input,output):
        N = input-output
        temp = output*N
        temp1 = data.shape[1]*(temp.sum(1))
        temp2 = (output.sum(1))*(N.sum(1))
        temp3 = torch.pow(data.shape[1]*((output**2).sum(1)) - ((output.sum(1))**2),1/2)
        temp4 = torch.pow(data.shape[1]*((N**2).sum(1))-((N.sum(1))**2),1/2) 
        loss = (temp1-temp2)/(temp3*temp4)

        a = torch.min(loss)
        b = torch.mean(1 - torch.exp(-0.5 * torch.pow((N / self.delta),2)))
        
        return self.r*a+(1-self.r)*b

In [17]:
class Loss0(nn.Module):
    def __init__(self,delta,r):
        super().__init__()
        self.delta = delta
        self.r = r
    def forward(self,input,output):
        N = input-output
        b = torch.mean(1 - torch.exp(-0.5 * torch.pow((N / self.delta),2)))
        K = output.shape[0]
        y2_mean = torch.mean(output)
        n2_mean = torch.mean(N)
        covariance = 0.0
        variance_y2 = 0.0
        variance_n2 = 0.0
        for j in range(K):
            y2_sample = output[j, :]
            n2_sample = N[j, :]
            covariance += ((y2_sample - y2_mean) * (n2_sample - n2_mean))
            variance_y2 += ((y2_sample - y2_mean) ** 2)
            variance_n2 += ((n2_sample - n2_mean) ** 2)
            lcc_value = (covariance / torch.sqrt(variance_y2 * variance_n2)) ** 2
            lcc_value = torch.mean(lcc_value)
        return self.r*lcc_value+(1-self.r)*b

In [18]:
device = torch.device("cuda") 
model = AutoEncoder(w1*w2,w1*w2).to(device)
#将模型转移到GPU上
#criterion = MeanHuberLoss(delta=1.3)
#criterion = WelschLoss(delta=0.5)
#criterion = Loss0(delta=0.46,r=0.05)#0.5 and 0.2,SNR:-8dB ok
criterion = Loss0(delta=0.42,r=0.01)
#criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [19]:
start_time = time.time()
es_cnt = 0
es_thres = 5
prev_train_loss = float('inf')
loss_train = []
loss_vaild = []
num_epochs = 100 # 总训练轮数
#num_batch_train = 0
for epoch in range(num_epochs):
  #train_bar = tqdm(train_loader)
  train_loss = 0.0
  
  for i , (batch) in enumerate(train_loader):

    # 数据转到device
    train_batch = batch[0].to(device)
    
    # 训练步骤  
    optimizer.zero_grad()
    outputs = model(train_batch)
    loss = criterion(outputs, train_batch)
    loss.backward() 
    optimizer.step()
    
    train_loss += loss.item()
    #num_batch_train +=1
  #train_loss除以所有bacth个数
  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
  loss_train.append(train_loss)
    



  # 验证
  valid_loss = 0.0
  #num_batch_vaild = 0
  with torch.no_grad():
    for i , (batch) in enumerate(vaild_loader):
    #for batch in vaild_loader:
    
      val_batch = batch[0].to(device)
      
      outputs = model(val_batch)
      loss = criterion(outputs, val_batch)
      valid_loss += loss.item()
      #num_batch_vaild += 1
    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
    loss_vaild.append(valid_loss)
    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))

    
    # Early stopping
    if train_loss - prev_train_loss >= 0:
        es_cnt += 1
    else:
        #es_cnt = 0
        pass

    if es_cnt >= es_thres:
        print(f"Early stopped at epoch {epoch}, train loss stop improving")
        break  

    prev_train_loss = train_loss
  print('loss_train: ', loss_train)
  print('vaild_train: ',loss_vaild)          
print("Training finished")
current_time = time.time()
time_sum = current_time-start_time
print(time_sum)

Epoch [1/100], Train Loss: 0.1740, Valid Loss: 0.1816
loss_train:  [0.173994489822043]
vaild_train:  [0.18155935077139965]
Epoch [2/100], Train Loss: 0.1593, Valid Loss: 0.1625
loss_train:  [0.173994489822043, 0.15933115083769142]
vaild_train:  [0.18155935077139965, 0.16249542239193732]
Epoch [3/100], Train Loss: 0.1542, Valid Loss: 0.1580
loss_train:  [0.173994489822043, 0.15933115083769142, 0.15416382411158228]
vaild_train:  [0.18155935077139965, 0.16249542239193732, 0.15801524098675984]
Epoch [4/100], Train Loss: 0.1514, Valid Loss: 0.1557
loss_train:  [0.173994489822043, 0.15933115083769142, 0.15416382411158228, 0.15139235685388727]
vaild_train:  [0.18155935077139965, 0.16249542239193732, 0.15801524098675984, 0.15568859731921783]
Epoch [5/100], Train Loss: 0.1504, Valid Loss: 0.1555
loss_train:  [0.173994489822043, 0.15933115083769142, 0.15416382411158228, 0.15139235685388727, 0.15036228956228279]
vaild_train:  [0.18155935077139965, 0.16249542239193732, 0.15801524098675984, 0.15568

In [20]:
loss_train = pd.DataFrame(loss_train)
loss_vaild = pd.DataFrame(loss_vaild)

loss = pd.concat([loss_train,loss_vaild],axis=1)

In [21]:
loss.columns = ['train_loss','vaild_loss']

In [22]:
torch.save(model.state_dict(), r'.\model_2dsyn11.pth')

In [23]:
model = AutoEncoder(w1*w2,w1*w2).to(device)
data = data.to(device)
model.load_state_dict(torch.load(r'.\model_2dsyn11.pth'))
model.eval()
with torch.no_grad():
    output = model(data)
    #loss = criterion(output, data)
print(output.shape)

torch.Size([33142, 2304])


In [24]:
output = output.cpu()
output = output.numpy()
output = pd.DataFrame(output)

In [25]:
loss.to_csv(r'loss_2dsyn11.csv',index=False)

In [26]:
output.to_csv(r'./output_2dsyn11.csv',index=None,header=None)

In [27]:
#loss_0 = []
#start_time = time.time()
#es_cnt = 0
#es_thres = 5
#prev_train_loss = float('inf')
#loss_train = []
#loss_vaild = []
#num_epochs = 100 # 总训练轮数
##num_batch_train = 0
#for epoch in range(num_epochs):
#  #train_bar = tqdm(train_loader)
#  train_loss = 0.0
#  
#  for i , (batch) in enumerate(train_loader):
#
#    # 数据转到device
#    train_batch = batch[0].to(device)
#    
#    # 训练步骤  
#    optimizer.zero_grad()
#    outputs = model(train_batch)
#    loss = criterion(outputs, train_batch)
#    loss.backward() 
#    optimizer.step()
#
#    loss_0.append(loss.item())
#
#    train_loss += loss.item()
#    #num_batch_train +=1
#  #train_loss除以所有bacth个数
#  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
#  loss_train.append(train_loss)
#    
#
#
#
#  # 验证
#  valid_loss = 0.0
#  #num_batch_vaild = 0
#  with torch.no_grad():
#    for i , (batch) in enumerate(vaild_loader):
#    #for batch in vaild_loader:
#    
#      val_batch = batch[0].to(device)
#      
#      outputs = model(val_batch)
#      loss = criterion(outputs, val_batch)
#      valid_loss += loss.item()
#      #num_batch_vaild += 1
#    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
#    loss_vaild.append(valid_loss)
#    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))
#
#    
#    # Early stopping
#    if train_loss - prev_train_loss >= 0:
#        es_cnt += 1
#    else:
#        #es_cnt = 0
#        pass
#
#    if es_cnt >= es_thres:
#        print(f"Early stopped at epoch {epoch}, train loss stop improving")
#        break  
#    
#
#
##   if epoch == 10:
##     torch.save(model.state_dict(), r'.\model_epochs10.pth')
##   elif epoch == 20:
##     torch.save(model.state_dict(), r'.\model_epochs20.pth')
##   elif epoch == 30:
##     torch.save(model.state_dict(), r'.\model_epochs30.pth')
##   elif epoch == 40:
##     torch.save(model.state_dict(), r'.\model_epochs40.pth')
##   elif epoch == 50:
##     torch.save(model.state_dict(), r'.\model_epochs50.pth')
##   elif epoch == 60:
##     torch.save(model.state_dict(), r'.\model_epochs60.pth')
##   elif epoch == 70:
##     torch.save(model.state_dict(), r'.\model_epochs70.pth')
##   elif epoch == 80:
##     torch.save(model.state_dict(), r'.\model_epochs80.pth')
##   elif epoch == 90:
##     torch.save(model.state_dict(), r'.\model_epochs90.pth')
##   elif epoch == 100:
##     torch.save(model.state_dict(), r'.\model_epochs100.pth')
##   else:
##       pass
#
#    prev_train_loss = train_loss
#  print('loss_train: ', loss_train)
#  print('vaild_train: ',loss_vaild)          
#print("Training finished")
#current_time = time.time()
#time_sum = current_time-start_time
#print(time_sum)